# Bim File

variant identifier and minor allele

In [1]:
import pandas as pd
df_bim=pd.read_csv('Data/0_Raw/DataS1.bim', sep='\t', header=None)
# df_bim = 1 and 4 columns
df_bim = df_bim[[1, 4]]

df_bim.rename(columns={1: 'VariantID', 4: 'minor'}, inplace=True)

In [2]:
# conver df_bim['variantID'] to a list
columns= df_bim['VariantID'].to_list()
columns.insert(0, 'familyID')
columns.insert(1, 'individualID')
columns.insert(2, 'fatherID')
columns.insert(3, 'motherID')
columns.insert(4,'Sex')
columns.insert(5, 'date')

In [3]:
minor=df_bim['minor'].to_list()
minor.insert(0, 'None')
minor.insert(1, 'None')
minor.insert(2, 'None')
minor.insert(3, 'None')
minor.insert(4,'None')
minor.insert(5, 'None')

# Ped File

familly id and individual id and allels

## Split the ped file by familly id

In [4]:
! rm -r Data/1_raw2freq/countries/*

rm: cannot remove 'Data/1_raw2freq/countries/*': No such file or directory


In [5]:
import os
 
def split_and_convert(file_path):
    # Create a directory to store output files
    os.makedirs('Data/1_raw2freq/countries', exist_ok=True)
    populations = []
    with open(file_path, 'r') as ped_file:
        for row in ped_file:
            row_data = row.strip().split()
            populations.append(row_data[0])
            pedigree = row_data[0]
            with open(f'Data/1_raw2freq/countries/{pedigree}.txt', 'a') as out_file:
                # Join the pairs of alleles in each row
                new_fields = row_data[:6]
                for i in range(6, len(row_data), 2):
                    new_fields.append(row_data[i] + row_data[i+1])
                # Write the new fields to the output file
                out_file.write(' '.join(new_fields) + '\n')
        
    return list(set(populations))





file_path = 'Data/0_Raw/DataS1.ped'

populations = split_and_convert(file_path)
print(populations)



['Israel', 'Bulgaria', 'Switzerland', 'Portugal', 'Germany', 'Latvia', 'Austria', 'Romania', 'Norway', 'Luxembourg', 'Estonia', 'UnitedKingdom', 'France', 'Poland', 'Turkey', 'Croatia', 'Jordan', 'Macedonia', 'Iran', 'Georgia', 'Greece', 'Sweden', 'Kazakhstan', 'Lithuania', 'Spain', 'TheNetherlands', 'Ukraine', 'Italy', 'Serbia', 'Hungary', 'Denmark', 'Armenia', 'CzechRepublic', 'Russia']


In [6]:
dfeuro = pd.read_excel(
    'Data/0_Raw/Eurasian - Dataset_tims.xlsx', sheet_name='Eurasian')
dfeuro = pd.DataFrame(dfeuro)
dfeuro.rename(columns={
              'Date mean in BP in years before 1950 CE [OxCal mu for a direct radiocarbon date, and average of range for a contextual date]': 'date'}, inplace=True)

# date_range is a list of the range of dates from 0 to 12000 years ago in 1000 year intervals
date_range = list(range(0, 12000, 1000)) 
# if dfeuro['date'] is in the range of date_range, then dfeuro['cat_date'] is the index of the range
dfeuro['cat_date']=pd.cut(dfeuro['date'], date_range, labels=range(0, len(date_range)-1))
# multiply the index by 1000 to get the range of dates
dfeuro['cat_date']=dfeuro['cat_date'].astype(int)*1000

dfeuro['cat_date']

0       5000
1       5000
2       4000
3       5000
4       5000
        ... 
4153    9000
4154    1000
4155    7000
4156    9000
4157    4000
Name: cat_date, Length: 4158, dtype: int64

In [7]:

def freq(pop_path, country):
    '''This function takes a population file and returns a dictionary of the frequency of each variant'''
    df = pd.read_csv(pop_path, sep=' ', header=None)
    df.columns = columns
    df.date = 'N'
    # print(len(df))
    # locate df['individualID'] in dfeuro['Master ID'] and add df['cat_date'] to the df[date]
    for i in range(0, len(df)):
        for j in range(0, len(dfeuro)):
            if df['individualID'][i] == dfeuro['Master ID'][j]:
                df['date'][i] = dfeuro['cat_date'][j]
  
    # get unique dates in df['date'] and store in a list and remove None
    date = df['date'].unique()
    date = date.tolist()
    if 'N' in date:
        date.remove('N')
    

    countryfreq = {}
    
    for d in date:
        freqdate= []
        df2 = df[df['date'] == d]
        for i in range(6, len(df.columns)):
            minorAllele = minor[i]
            count = 0
            total = len(df2)*2
            for j in range(0, len(df2)):
                # count occurrences of the minor allele in column i
                count += str(df2.iloc[j, i]).count(str(minorAllele))
            freqdate.append(count/total)
            
        name= f'{country}_{d}'
        countryfreq[name] = freqdate
        
    return countryfreq





In [8]:
import multiprocessing
! rm Data/S1_freq.csv
def process_country(country):
    pop_path = f'Data/1_raw2freq/countries/{country}.txt'
    freq_results = freq(pop_path, country)
    with open('Data/1_raw2freq/S1_freq.csv', 'a') as f:
        for key, value in freq_results.items():
            countr = key.split('_')[0]
            datee= key.split('_')[1]
            str_values = [str(val) for val in value]  # convert float values to strings
            f.write('%s\t%s\t%s\n' % (countr, datee,'\t'.join(str_values)))

if __name__ == '__main__':
    with multiprocessing.Pool() as pool:
        pool.map(process_country, populations)

/tmp/ipykernel_15275/2522827221.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'][i] = dfeuro['cat_date'][j]


## Filter SNPs which have 0 in more than 80% of the countries-age groups

In [10]:
import pandas as pd
df = pd.read_csv('Data/1_raw2freq/S1_freq.csv', sep='\t', header=None)
#firts coumn name is country and from second to last column name is columns[5:]



In [11]:
df

,0,1,2,3,4,5,6,7,8,9,...,147221,147222,147223,147224,147225,147226,147227,147228,147229,147230
0,Macedonia,7000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Portugal,4000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,...,0.333333,0.0,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.0
2,Estonia,6000,0.500000,0.000000,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,...,0.500000,0.0,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0
3,Estonia,5000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Austria,7000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Hungary,4000,0.000000,0.090909,0.181818,0.181818,0.000000,0.090909,0.363636,0.000000,...,0.090909,0.0,0.090909,0.090909,0.272727,0.000000,0.000000,0.090909,0.000000,0.0
115,Hungary,6000,0.086957,0.086957,0.260870,0.260870,0.173913,0.086957,0.173913,0.043478,...,0.086957,0.0,0.043478,0.217391,0.086957,0.086957,0.130435,0.000000,0.000000,0.0
116,Hungary,5000,0.058824,0.000000,0.411765,0.000000,0.000000,0.117647,0.058824,0.058824,...,0.058824,0.0,0.058824,0.058824,0.235294,0.000000,0.117647,0.000000,0.058824,0.0
117,Hungary,7000,0.054054,0.081081,0.405405,0.081081,0.081081,0.027027,0.108108,0.054054,...,0.081081,0.0,0.081081,0.108108,0.108108,0.054054,0.081081,0.000000,0.000000,0.0


In [12]:
df.columns = ['country', 'date'] + df_bim['VariantID'].to_list()
df

,country,date,rs3094315,rs6696609,rs9442372,rs4970405,rs11807848,rs4970421,rs2887286,rs113791678,...,rs739365,rs148828263,rs9616945,rs5770820,rs6010062,rs715586,rs8137951,rs3810648,rs78827609,rs116026001
0,Macedonia,7000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Portugal,4000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,...,0.333333,0.0,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.0
2,Estonia,6000,0.500000,0.000000,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,...,0.500000,0.0,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0
3,Estonia,5000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Austria,7000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Hungary,4000,0.000000,0.090909,0.181818,0.181818,0.000000,0.090909,0.363636,0.000000,...,0.090909,0.0,0.090909,0.090909,0.272727,0.000000,0.000000,0.090909,0.000000,0.0
115,Hungary,6000,0.086957,0.086957,0.260870,0.260870,0.173913,0.086957,0.173913,0.043478,...,0.086957,0.0,0.043478,0.217391,0.086957,0.086957,0.130435,0.000000,0.000000,0.0
116,Hungary,5000,0.058824,0.000000,0.411765,0.000000,0.000000,0.117647,0.058824,0.058824,...,0.058824,0.0,0.058824,0.058824,0.235294,0.000000,0.117647,0.000000,0.058824,0.0
117,Hungary,7000,0.054054,0.081081,0.405405,0.081081,0.081081,0.027027,0.108108,0.054054,...,0.081081,0.0,0.081081,0.108108,0.108108,0.054054,0.081081,0.000000,0.000000,0.0


In [17]:
dff=[]
for i in range(2, len(df.columns)):
   # count the number of times a value == 0 in each column
    count = df.iloc[:, i].value_counts()[0]
    dff.append(count)
    
# add zero in index 0 and 1 of dff
dff.insert(0, 0)
dff.insert(1, 0)
dff

[0,
 0,
 83,
 61,
 39,
 81,
 81,
 106,
 69,
 92,
 112,
 111,
 117,
 117,
 80,
 57,
 78,
 104,
 53,
 52,
 57,
 85,
 52,
 59,
 110,
 119,
 113,
 111,
 88,
 59,
 118,
 40,
 110,
 119,
 60,
 61,
 71,
 119,
 49,
 119,
 56,
 67,
 66,
 112,
 117,
 90,
 68,
 50,
 72,
 118,
 51,
 67,
 59,
 119,
 119,
 117,
 119,
 42,
 47,
 114,
 62,
 113,
 50,
 87,
 98,
 117,
 74,
 62,
 84,
 116,
 111,
 47,
 95,
 48,
 88,
 47,
 69,
 91,
 86,
 74,
 77,
 86,
 105,
 82,
 105,
 66,
 74,
 42,
 100,
 44,
 79,
 119,
 51,
 119,
 99,
 54,
 106,
 85,
 57,
 65,
 86,
 72,
 82,
 110,
 103,
 119,
 67,
 65,
 70,
 53,
 119,
 63,
 90,
 66,
 69,
 119,
 52,
 93,
 86,
 64,
 63,
 109,
 74,
 39,
 115,
 117,
 111,
 55,
 72,
 81,
 67,
 67,
 107,
 54,
 55,
 112,
 79,
 102,
 112,
 91,
 46,
 60,
 76,
 76,
 92,
 57,
 119,
 70,
 51,
 99,
 54,
 51,
 112,
 71,
 96,
 119,
 72,
 119,
 51,
 103,
 67,
 54,
 37,
 92,
 70,
 118,
 118,
 87,
 70,
 104,
 64,
 67,
 51,
 62,
 75,
 119,
 119,
 61,
 88,
 83,
 97,
 84,
 82,
 79,
 88,
 74,
 68,
 94,
 118,


In [22]:
df2=df.copy()
df2

,country,date,rs3094315,rs6696609,rs9442372,rs4970405,rs11807848,rs4970421,rs2887286,rs113791678,...,rs739365,rs148828263,rs9616945,rs5770820,rs6010062,rs715586,rs8137951,rs3810648,rs78827609,rs116026001
0,Macedonia,7000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,Portugal,4000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,...,0.333333,0.0,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.0
2,Estonia,6000,0.500000,0.000000,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,...,0.500000,0.0,0.500000,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.0
3,Estonia,5000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,Austria,7000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Hungary,4000,0.000000,0.090909,0.181818,0.181818,0.000000,0.090909,0.363636,0.000000,...,0.090909,0.0,0.090909,0.090909,0.272727,0.000000,0.000000,0.090909,0.000000,0.0
115,Hungary,6000,0.086957,0.086957,0.260870,0.260870,0.173913,0.086957,0.173913,0.043478,...,0.086957,0.0,0.043478,0.217391,0.086957,0.086957,0.130435,0.000000,0.000000,0.0
116,Hungary,5000,0.058824,0.000000,0.411765,0.000000,0.000000,0.117647,0.058824,0.058824,...,0.058824,0.0,0.058824,0.058824,0.235294,0.000000,0.117647,0.000000,0.058824,0.0
117,Hungary,7000,0.054054,0.081081,0.405405,0.081081,0.081081,0.027027,0.108108,0.054054,...,0.081081,0.0,0.081081,0.108108,0.108108,0.054054,0.081081,0.000000,0.000000,0.0


In [23]:

# get indeces of dff where that are > 80
index = [i for i, x in enumerate(dff) if x > 95]
# remove df2 columns that are in index
df2.drop(df2.columns[index], axis=1, inplace=True)
df2.shape

(119, 86687)

In [24]:
df2

,country,date,rs3094315,rs6696609,rs9442372,rs4970405,rs11807848,rs2887286,rs113791678,rs3843249,...,rs12160714,rs131749,rs76610029,rs140513,rs739365,rs9616945,rs5770820,rs6010062,rs715586,rs8137951
0,Macedonia,7000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Portugal,4000,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.333333,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.333333,0.000000
2,Estonia,6000,0.500000,0.000000,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.500000,0.000000,0.000000,0.500000
3,Estonia,5000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Austria,7000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.500000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,Hungary,4000,0.000000,0.090909,0.181818,0.181818,0.000000,0.363636,0.000000,0.181818,...,0.090909,0.454545,0.000000,0.181818,0.090909,0.090909,0.090909,0.272727,0.000000,0.000000
115,Hungary,6000,0.086957,0.086957,0.260870,0.260870,0.173913,0.173913,0.043478,0.000000,...,0.043478,0.304348,0.130435,0.304348,0.086957,0.043478,0.217391,0.086957,0.086957,0.130435
116,Hungary,5000,0.058824,0.000000,0.411765,0.000000,0.000000,0.058824,0.058824,0.058824,...,0.000000,0.117647,0.058824,0.294118,0.058824,0.058824,0.058824,0.235294,0.000000,0.117647
117,Hungary,7000,0.054054,0.081081,0.405405,0.081081,0.081081,0.108108,0.054054,0.027027,...,0.000000,0.243243,0.108108,0.270270,0.081081,0.081081,0.108108,0.108108,0.054054,0.081081


In [25]:
# save to csv without index and header
# df2.to_csv('Data/1_raw2freq/S1_freq_filtered.csv', index=False, header=False, sep='\t')

df2.to_csv('Data/1_raw2freq/maf_filteered.csv', index=False, header=False, sep='\t')


In [38]:
# convert every *.csv file in the directory to *.csv.gz
! gzip -9 Data/1_raw2freq/*.csv
